In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead, GPT2Config
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import json
import random
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

In [7]:
# Load the JSON file
with open('../data/finalWords.json') as f:
    words = json.load(f)


#model_name = 'gpt2'
model_path = "../models/random-model"
tokenizer_path = "../models/random-token"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
config_path = "../models/random-model/config.json"
config = GPT2Config.from_json_file(config_path)
model = GPT2DoubleHeadsModel.from_pretrained(model_path, config = config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# get new words
with open('../data/finetune_eval.txt', 'r') as file:
    lines = file.readlines()

new_words = [word.strip() for line in lines for word in line.split()]
print(len(new_words))

7501


In [9]:
# Initialize the quiz questions and answers
quiz_questions = []
quiz_answers = []
new_words_copy = new_words[:]
# Loop through each word in the JSON file
count = 0

for word in words:
    if word['word'] in new_words:
        new_words.remove(word['word'])
        example = word['example']
        # Get a list of 3 random words to use as distractors
        distractors = random.sample(new_words_copy, 4)
        # Add the correct answer to the beginning of the options list
        options = [word['example'] + " [CLS]"] + [example.replace(word['word'], word_choice) + " [CLS]" for word_choice in distractors]
        # Add the question and set the answer to 0
        quiz_questions.append(options)
        quiz_answers.append(0)
        count += 1
        if count % 100 == 0:
            print(count)
print(quiz_questions[0])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
["We be dubbin! And we's gonna bounce to play pool later... Yayeeyay! [CLS]", "We be dubbin! And we's gonna bounce to play pool later... Yayeeyay! [CLS]", "We be dubbin! And we's gonna bounce to play pool later... Yayeeyay! [CLS]", "We be dubbin! And we's gonna bounce to play pool later... Yayeeyay! [CLS]", "We be dubbin! And we's gonna bounce to play pool later... Yayeeyay! [CLS]"]


In [10]:
print(len(new_words))

0


In [11]:
# Add a [CLS] to the vocabulary (we should train it also!)
num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
embedding_layer = model.resize_token_embeddings(len(tokenizer))

questions = quiz_questions
answers = quiz_answers

count0 = 0.0
count1 = 0.0
count2 = 0.0
count3 = 0.0
count4 = 0.0
total = 0.0
total = 0.0
for i, choices in enumerate(questions):
    #print(choices)
    encoded_choices = [tokenizer.encode(s) for s in choices]
    cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
    #check if some choices are longer than the others
    max_len = max([len(tokens) for tokens in encoded_choices])
    check = True
    for tokens in encoded_choices:
        if len(tokens) < max_len:
            check = False
    if check:
        input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
        mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
        outputs = model(input_ids, mc_token_ids=mc_token_ids)
        lm_logits = outputs.logits
        mc_logits = outputs.mc_logits
        if mc_logits.argmax().item() == 0:
            count0 += 1
        elif mc_logits.argmax().item() == 1:
            count1 += 1
        elif mc_logits.argmax().item() == 2:
            count2 += 1
        elif mc_logits.argmax().item() == 3:
            count3 += 1
        elif mc_logits.argmax().item() == 4:
            count4 += 1
        
        total += 1

print(len(questions))
print(count0)
print(count1)
print(count2)
print(count3)
print(count4)
print(total)

7501
2781.0
574.0
629.0
555.0
1425.0
5964.0
